In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report


---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [2]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
df = pd.read_csv(
    Path('Resources/lending_data.csv')   
)

# Review the DataFrame
df.head()


,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [3]:
# Separate the data into labels and features
y = df['loan_status']
X = df.drop(columns='loan_status')


In [4]:
# Review the y variable Series
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [5]:
# Review the X variable DataFrame
X.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [6]:
# Check the balance of our target values
label_count = pd.value_counts(y)
print(label_count)

0    75036
1     2500
Name: loan_status, dtype: int64


### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [7]:
# Import the train_test_learn module
from sklearn.model_selection import train_test_split

# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [8]:
# Import the LogisticRegression module from SKLearn
from sklearn.linear_model import LogisticRegression

# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
logistic_regression_model = LogisticRegression(random_state=1)

# Fit the model using training data
lr_model = logistic_regression_model.fit(X_train, y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [9]:
# Make a prediction using the testing data
testing_predictions = lr_model.predict(X_test)


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [10]:
# Print the balanced_accuracy score of the model
accuracy_score = balanced_accuracy_score(y_test, testing_predictions)

In [11]:
# Generate a confusion matrix for the model
cm = confusion_matrix(y_test, testing_predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)


In [12]:
# Print the classification report for the model
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {accuracy_score}")
print("Classification Report")
print(classification_report(y_test,testing_predictions))

Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,18663,102
Actual 1,56,563


Accuracy Score : 0.9520479254722232
Classification Report
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** 
In short, the logistic regression model performs very well in predicting healthy loans (label 0) and reasonably well in identifying high-risk loans (label 1), although there is potential for further refinement to improve its performance on the latter.

---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [13]:
# Import the RandomOverSampler module form imbalanced-learn
from imblearn.over_sampling import RandomOverSampler


# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model

random_oversampler = RandomOverSampler(random_state=1)
# Fit the original training data to the random_oversampler model
X_train_resampled, y_train_resampled = random_oversampler.fit_resample(X_train, y_train)


In [33]:
X_resampled, y_resampled = random_oversampler.fit_resample(X, y)


In [35]:
X_train_resampled, X_test_resampled, y_train_resampled, y_test_resampled = train_test_split(X_resampled, y_resampled)

In [20]:
# Count the distinct values of the resampled labels data
unique_labels = np.unique(y_train_resampled)
num_unique_labels = len(unique_labels)

print("Number of distinct labels in the resampled data:", num_unique_labels)

Number of distinct labels in the resampled data: 2


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [37]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
lr_model1 = LogisticRegression(random_state=1)

# Fit the model using the resampled training data
lr_model1.fit(X_train_resampled, y_train_resampled)

# Make a prediction using the testing data
y_pred = lr_model1.predict(X_test_resampled)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [38]:
# Print the balanced_accuracy score of the model 
balanced_accuracy_score(y_test_resampled, y_pred)

0.9942963644606917

In [39]:
# Generate a confusion matrix for the model
confusion_matrix(y_test_resampled,y_pred)

array([[18686,   110],
       [  104, 18618]])

In [41]:
# Print the classification report for the model
print(classification_report(y_test_resampled, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     18796
           1       0.99      0.99      0.99     18722

    accuracy                           0.99     37518
   macro avg       0.99      0.99      0.99     37518
weighted avg       0.99      0.99      0.99     37518



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** Precision: Precision measures the accuracy of the positive predictions made by the model. For both labels, the precision is 0.99, indicating that the model has a high percentage of correct predictions for both healthy and high-risk loans.
Recall: Recall, also known as sensitivity, measures the proportion of actual positives that are correctly identified by the model. For both labels, the recall is 0.99, indicating that the model effectively captures most of the positive instances for both healthy and high-risk loans.
F1-score: The F1-score is the harmonic mean of precision and recall, providing a balanced measure of model performance. For both labels, the F1-score is 0.99, indicating excellent overall performance in predicting both healthy and high-risk loans.
Support: The support refers to the number of instances of each class in the dataset. For the "0" label (healthy loan), there are 18,796 instances, and for the "1" label (high-risk loan), there are 18,722 instances.
Overall, the model achieves an accuracy of 0.99, indicating that it accurately predicts the majority of instances for both healthy and high-risk loans. The macro average and weighted average of precision, recall, and F1-score are also 0.99, demonstrating consistent and high-quality predictions for both labels.